In [8]:
import pandas as pd
import numpy as np
import jqdatasdk as jq
import os

In [9]:
jq.auth('13918852005','960312Lsc')
# jq.auth('15821912507','912507')

In [10]:
stock_download_lst = pd.read_csv('E:/Stock_Data/stock_code.csv')
stock_download_lst = list(stock_download_lst.iloc[:,0])
stock_download_len = len(stock_download_lst)

In [11]:
trade_days = jq.get_trade_days(start_date='2011-01-01', end_date='2018-12-31')
trade_days_len = len(trade_days)

In [12]:
used_stock_list = []
cnt_code = 0
code_list_temp = os.listdir('E:/Stock_Data/stock_return_data')
code_list = [i[:-4] for i in code_list_temp]

for code in stock_download_lst:
    
    if code in code_list:
        cnt_code += 1
        print('No.'+str(cnt_code)+" Complete:%.4f%%" % (cnt_code/stock_download_len*100),end="\r")
        continue
    
    price_inf = jq.get_price(code, 
                 start_date='2011-01-01', 
                 end_date='2018-12-31', 
                 frequency='daily', 
                 fields=['close'], 
                 skip_paused=False, 
                 fq='pre', 
                 count=None)
    
    # 价格中出现零，剔除
    price_zero_num = len(price_inf[price_inf['close']==0])
    if price_zero_num != 0:
        continue
    
    price_inf['close_1'] = price_inf.shift(1)
    price_inf['return'] = np.log(price_inf['close']/price_inf['close_1'])
    price_inf = price_inf.dropna(how="any",axis=0)
    price_save_df = price_inf[['return']]
    
    # 记录收益率为0的天数和，停牌过多，剔除
    return_zero_num = len(price_save_df[price_save_df['return']==0])
    if return_zero_num > trade_days_len/3.8: # 2年的时间都处于停牌状态，剔除
        continue

    # 保存
    cnt_code += 1
    used_stock_list.append(code)
    price_save_df.to_csv('E:/Stock_Data/stock_return_data/'+code+'.csv',index=True)
    print('No.'+str(cnt_code)+" Complete:%.4f%%" % (cnt_code/stock_download_len*100),end="\r")
    
pd.DataFrame(used_stock_list).to_csv('E:/Stock_Data/return_inf_used_stock.csv',index=False)

In [13]:
price_save_df.to_csv('E:/Stock_Data/stock_return_data/'+code+'.csv',index=True)
pd.DataFrame(used_stock_list).to_csv('E:/Stock_Data/return_inf_used_stock.csv',index=False)